### BIM to GIS Conversion

#### This Notebook imports revit files from BIM models and convert them to GIS geodatabase in prepartion for monitoring
#### Please implement the following steps:
1. Define Projection to PRS92
2. Export Feature to file geodatabase
3. Merge all feature layers
4. Delete unwanted fields
5. Add Fields
6. Calculate Field for Type
7. Calculate Field for SubType
8. Delete Fields

In [71]:
import time, os, datetime, sys, logging, logging.handlers, shutil, traceback, re
import arcpy

arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("WGS 1984 Web Mercator (auxiliary sphere)")
arcpy.env.geographicTransformations = "PRS_1992_To_WGS_1984_1"
workspace = r'C:/Users/emasu/OneDrive - Oriental Consultants Global JV/Documents/ArcGIS/Projects/During-Construction_nscrexn2/During-Construction_nscrexn2.gdb'


In [72]:
# First, makre sure to Add to Current Map all the models:
## e.g. 'StructuralColumns', 'Roofs'

# 0. Station name
station = "Apalit"

if station == "CIA":
    fDataset = r'Station_BG'
else:
    fDataset = r'Station_AG'
    
path = os.path.join(workspace, fDataset)

In [73]:
# 1. Define Projection to PRS92
## Add revit files to the Contents panel
## Note that 'Floors' in the BIM Model includes both slab and floors together.

columnA = r'Columns'
floor = r'Floors'
roof = r'Roofs'
framing = r'StructuralFraming'
foundation = r'StructuralFoundation'
column = r'StructuralColumns'

# Note that if we import from '-MOD-A-000001.rvt', it uses different 'BldgLevel' domain
# names and levels fro '-MOD-ST-000001.rvt'.
# It may be better for the GIS model to only utilize '-MOD-ST-000001.rvt' (i.e., drop 'roof')

if station == "Apalit" or station == "CIA":
    in_features = [columnA, floor, framing, foundation, column]
elif station == "Clark":
    in_features = [floor, framing, foundation, column]


sr = arcpy.SpatialReference("PRS 1992 Philippines Zone III")

for fl in in_features:
    # Define Projection first
    arcpy.DefineProjection_management(fl, sr)    


In [74]:
# 2. Export to file geodatabase

arcpy.FeatureClassToGeodatabase_conversion(in_features, path)


<Result 'C:\\Users\\emasu\\OneDrive - Oriental Consultants Global JV\\Documents\\ArcGIS\\Projects\\During-Construction_nscrexn2\\During-Construction_nscrexn2.gdb\\Station_AG'>

In [76]:
# 3. Merge all feature layers
arcpy.env.workspace = path
output_feature = "Station_Structure_merged" + "_" + station
arcpy.Merge_management(in_features, output_feature,"","")

<Result 'C:/Users/emasu/OneDrive - Oriental Consultants Global JV/Documents/ArcGIS/Projects/During-Construction_nscrexn2/During-Construction_nscrexn2.gdb\\Station_AG\\Station_Structure_merged_Apalit'>

In [77]:
# 4. Delete Fields
## 4.1 Obtain fields to be kept
fieldList = [f.name for f in arcpy.ListFields(output_feature)]

# find OBJECTID, as this is not deletable
reg = re.compile(r'OBJECTID*|Shape|Shape_Length|Shape_Area|Shape.STArea()|Shape.STLength()|GlobalID')
objField = list(filter(reg.match, fieldList))
keepFields = objField + ['Category', 'Family', 'FamilyType','ObjectID', 'ObjectId', 'Bldg_Name', 'BldgLevel', 'OmniClassDescription', 'BldgLevel_Desc', 'DocName', 'Discipline']

# Drop fields
dropFields = [e for e in fieldList if e not in keepFields]

## 4.2 Delete fields from feature layer
arcpy.DeleteField_management(output_feature, dropFields)

<Result 'Station_Structure_merged_Apalit'>

In [78]:
# 5. Add Fields
addFields = ['Station', 'Type', 'SubType', 'Status', 'StartDate', 'TargetDate', 'StructureType', 'StructureLevel']

for field in addFields:
    if field in ('StartDate', 'TargetDate'):
        arcpy.AddField_management(output_feature, field, "DATE", "", "", "", field, "NULLABLE", "")
    else:
        arcpy.AddField_management(output_feature, field, "SHORT", "", "", "", field, "NULLABLE", "")

In [70]:
# Each station may have different criteria
# 6. Add "temp" field 
## Some Subtypes (domain) are not automatically extractable from existing information.
## As such, we must extract matching letters for these SubTypes.
arcpy.AddField_management(output_feature, 'temp', "TEXT", "", "", "", 'temp', "NULLABLE","")

############ APALIT STATION ###############
# if station == 'Apalit':
FGirder = ['FG1','FG2','FG3','FG4','FG5','FG11','FG12','FG21']
FBeam = ['B100','B150']
Columns = ['SC1','SC2','2000x2000'] # 2000x2000 is from architectural column
FireExitStair = ['MC13X50']
SteelStaircase = ['W18X106']
BeamConcourse = ['B70','B70A','B90','B100','B150']
BeamPlatform = ['SB40','SB40A','SB40B','SB50','SB60','SB60M','SB100A']
SlabConcourse = ['S20','S25','S25A']
SlabPlatform = ['S15']
Girder = ['PSG']
RoofGirder = ['RSG']
Brace = ['HB1']
RoofBeam = ['SB30M','SB50M','SB70N','SB85']
Delete = ['TB1_','B1_','C1_','S1_','S2_','HSS','W8X31','W8X15', 'W12X26']
# 'W12X26' is Slab Beam on steel staircase

joinText = FGirder + FBeam + Columns + FireExitStair + \
             SteelStaircase + BeamConcourse + \
             BeamPlatform + SlabConcourse + SlabPlatform + \
             Girder + RoofGirder + Brace + RoofBeam + Delete

searchText = "|".join(joinText)

with arcpy.da.UpdateCursor(output_feature, ["FamilyType","temp"]) as cursor:
    for row in cursor:
        if row[0]:
            try:
                reg = re.search(searchText,row[0]).group()
            except AttributeError:
                reg = re.search(searchText,row[0])
            row[1] = reg
        else:
            row[1] = None
        cursor.updateRow(row)

In [69]:
## APALIT

# Delete rows if FamilyType includes 'TB1_|B1_|C1_|S1_|S2_|HSS|W8X|W8X'
with arcpy.da.UpdateCursor(output_feature, ["temp"]) as cursor:
    for row in cursor:
        if row[0] in Delete:
            cursor.deleteRow()

In [44]:
# Enter 'Type' and 'SubType'
# 'Type' field domain:
## 1. Foundations
## 2. Columns
## 3. Framing
## 4. Roofs
## 5. Floor

# 'SubType' field domain:
## 1. Pile
## 2. Pile Cap
## 3. Column
## 4. Foundation Girder []
## 5. Girder
## 6. Roof Girder
## 7. Foundation Beam
## 8. Beam
## 9. Brace
## 10. Fire Exit Stair
## 11. Steel Staircase
## 12. Slab
## 13. Roof Beam

FGirder = ['FG1','FG2','FG3','FG4','FG5','FG11','FG12','FG21']
FBeam = ['B100','B150']
Columns = ['SC1','SC2','2000x2000'] # 2000x2000 is from architectural column
FireExitStair = ['MC13X50']
SteelStaircase = ['W18X106']
BeamConcourse = ['B70','B70A','B90','B100','B150']
BeamPlatform = ['SB40','SB40A','SB40B','SB50','SB60','SB60M','SB100A']
SlabConcourse = ['S20','S25','S25A']
SlabPlatform = ['S15']
Girder = ['PSG']
RoofGirder = ['RSG']
Brace = ['HB1']
RoofBeam = ['SB30M','SB50M','SB70N','SB85']
Delete = ['TB1_','B1_','C1_','S1_','S2_','HSS','W8X31','W8X15']

with arcpy.da.UpdateCursor(output_feature, ['Category', 'Type', 'SubType','FamilyType', 'OmniClassDescription','temp']) as cursor:
    for row in cursor:
        if row[0] == 'Structural Foundations' and row[4] == 'Foundation Piles': # Pile and Pile Cap as one
            row[1] = 1
            row[2] = 1
        elif row[4] == 'Columns': # Columns
            row[1] = 2
            row[2] = 3
        elif row[5] in FGirder: # Foundation Girder
            row[1] = 3
            row[2] = 4
        elif row[5] == Girder: # Girder
            row[1] = 3
            row[2] = 5
        elif row[5] == RoofGirder: # Roof Girder
            row[1] = 3
            row[2] = 6
        elif row[5] in FBeam: # Foundation Beam
            row[1] = 3
            row[2] = 7
        elif row[5] in (BeamConcourse, BeamPlatform): # Beam
            row[1] = 3
            row[2] = 8
        elif row[5] == Brace: # Brace
            row[1] = 3
            row[2] = 9
        elif row[5] == FireExitStair: # Fire Exit Stair
            row[1] = 3
            row[2] = 10
        elif row[5] == SteelStaircase: # Steel Staircase
            row[1] = 3
            row[2] = 11
        elif row[0] == 'Floors':
            row[1] = 5
            row[2] = 12
        elif row[0] in RoofBeam: # Roof Beam
            row[1] = 3
            row[2] = 13
        else:
            row[1] = None
            row[2] = None
        #elif row[0] == 'Roofs':
        #    row[1] = 4
        #    row[2] = 5
        cursor.updateRow(row)

TypeError: 'in <string>' requires string as left operand, not NoneType

In [ ]:
# Enter 'StructureType' and 'StructureLevel' from P6
# StructureType
## 1. Substructure
## 2. Superstructure

# StructureLevel
## 1. Foundation
## 2. Concourse Level
## 3. Platform Level
## 4. Roof Level

# 'SubType' field domain:
## 1. Pile
## 2. Pile Cap
## 3. Column
## 4. Foundation Girder []
## 5. Girder
## 6. Roof Girder
## 7. Foundation Beam
## 8. Beam
## 9. Brace
## 10. Fire Exit Stair
## 11. Steel Staircase
## 12. Slab
## 13. Roof Beam

with arcpy.da.UpdateCursor(output_feature, ['SubType', 'temp', 'StructureType', 'StructureLevel']) as cursor:
    for row in cursor:
        # 'Foundation' and 'Substructure'
        if row[0] in (FGirder, FBeam): 
            row[2] = 1
            row[3] = 1
        # Concorse Level
        elif row[0] in (Columns, FireExitStair, SteelStaircase): # 'Concourse Level'
            row[3] = 2
        elif row[1] in BeamConcourse:
            row[3] = 2
        elif row[1] in SlabConcourse:
            row[3] = 2
        # Platform Level
        elif row[1] in (BeamPlatform, Girder, SlabPlatform):
            row[3] = 3
        # Roof Level
        elif row[1] in  (RoofGirder, Brace, RoofBeam):
            row[3] = 4
        # SuperStructure
        elif row[0] not in (FGirder, FBeam):
            row[2] = 2
        

In [26]:
# Delete observations with SubType = None
with arcpy.da.UpdateCursor(output_feature, ['SubType']) as cursor:
    for row in cursor:
        if row[0] is None:
            cursor.deleteRow()

In [ ]:
# Delete some which do not require monitoring
## Apalit
### FamilyType includes the values 'W8X31', 'W8X15', 'HSS', 'TB1'
#with arcpy.da.UpdateCursor(output_feature, ['Station', 'FamilyType']) as cursor:
#    for row in cursor:
#        if row[0] == 7 and row[1]: # Apalit
            
        

In [27]:
# Set the initial status = 'To be Constructed'
with arcpy.da.UpdateCursor(output_feature, ['Status']) as cursor:
    for row in cursor:
        row[0] = 1
        cursor.updateRow(row)

In [28]:
# Enter station name
with arcpy.da.UpdateCursor(output_feature, ['Station']) as cursor:
    for row in cursor:
        if station == "NCC":
            row[0] = 1
        elif station == "Depot":
            row[0] = 2
        elif station == "CIA":
            row[0] = 3
        elif station == "Clark":
            row[0] = 4
        elif station == "Angeles":
            row[0] = 5
        elif station == "San Fernando":
            row[0] = 6
        elif station == "Apalit":
            row[0] = 7
        elif station == "Calumpit":
            row[0] = 8
        elif station == "Malolos":
            row[0] = 9
        elif station == "Solis":
            row[0] = 10
        elif station == "Blumentritt":
            row[0] = 11
        elif station == "Espana":
            row[0] = 12
        elif station == "Santa Mesa":
            row[0] = 13
        elif station == "Paco":
            row[0] = 14
        elif station == "Buendia":
            row[0] = 15
        elif station == "EDSA":
            row[0] = 16
        elif station == "Nichols":
            row[0] = 17
        elif station == "FTI":
            row[0] = 18
        elif station == "Bicutan":
            row[0] = 19
        elif station == "Sucat":
            row[0] = 20
        elif station == "Alabang":
            row[0] = 21
        elif station == "Muntinlupa":
            row[0] = 22
        elif station == "San Pedro":
            row[0] = 23
        elif station == "Pacita":
            row[0] = 24
        elif station == "Binan":
            row[0] = 25
        elif station == "Santa Rosa":
            row[0] = 26
        elif station == "Cabuyao":
            row[0] = 27
        elif station == "Banlic Depot":
            row[0] = 28
        elif station == "Banlic":
            row[0] = 29
        elif station == "Calamba":
            row[0] = 30
        cursor.updateRow(row)


In [29]:
# Assign Domains
# 0. Domain Name
domainList = ['Station Structures_TYPE', 'Station Structures_SUBTYPE',\
              'Station Structures_STATUS', 'Station Structures_STRUCTURETYPE',\
              'Station Structures_STRUCTURELEVEL','Station_nscrex']

#domains = arcpy.da.ListDomains(workspace)
#listDomain = [d.name for d in domains]
#reg = re.compile(r'Station Structures*|Station*')
#listFinal = list(filter(reg.match, listDomain))

arcpy.AssignDomainToField_management(output_feature, 'Type', domainList[0])
arcpy.AssignDomainToField_management(output_feature, 'SubType', domainList[1])
arcpy.AssignDomainToField_management(output_feature, 'Status', domainList[2])
arcpy.AssignDomainToField_management(output_feature, 'StructureType', domainList[3])
arcpy.AssignDomainToField_management(output_feature, 'StructureLevel', domainList[4])
arcpy.AssignDomainToField_management(output_feature, 'Station', domainList[5])

<Result 'Station_Structure_merged_Apalit'>

In [30]:
# Apply Symbology
symbolLyrx = r'C:/Users/emasu/Dropbox/01-Railway/02-NSCR-Ex/10-ArcGIS/01-Reference/01-Layer File/03-Station_Structure/Station_Structures.lyrx'
arcpy.ApplySymbologyFromLayer_management(output_feature, symbolLyrx, [["VALUE_FIELD", "Status", "Status"]], update_symbology="MAINTAIN")[0]

In [31]:
# Alter field. Rename a field name 'ObjectId' to 'ObjectId1'. The GIS table has OBJECTID, which is reserved name.
# We cannot use this name.
arcpy.AlterField_management(output_feature, 'ObjectId', 'ObjectId1', 'ObjectId1', 'TEXT')


<Result 'Station_Structure_merged_Apalit'>

In [32]:
# Delete Fields
dropF = ['Bldg_Name', 'OmniClassDescription', 'temp']
arcpy.DeleteField_management(output_feature, dropF)

<Result 'Station_Structure_merged_Apalit'>

In [33]:
# Add 'Grid' and 'Zone' for monitoring. this is used in reference to P6, too
addFields = ['Grid', 'Zone']

for field in addFields:
    arcpy.AddField_management(output_feature, field, "TEXT", "", "", "", field, "NULLABLE","")